In [ ]:
# Oh maybe we could just pick out each of the samples from each of them and use that...Then also extend to the other datasets.

# So yeah - let's feed ChatGPT and see what scripts it could generate for me

In [30]:
import json

# Step 1: Read both JSON files and load the data into memory
with open('../spider/train_spider.json') as f:
    spiderDataPoints = json.load(f)

with open('../sql_create_context_v4.json') as f:
    sqlCreateContextPoints = json.load(f)

In [31]:
print(len(spiderDataPoints))

7000


In [32]:
import sqlparse

def normalize_query(query):
    # Lowercase the query
    query = query.lower()
    
    # Parse and format the query to normalize the spacing
    parsed_query = sqlparse.parse(query)
    normalized_query = " ".join([sqlparse.format(str(stmt), strip_comments=True, strip_whitespace=True) for stmt in parsed_query])
    
    return normalized_query

# Test the function
query = "SELECT count(*) FROM pets WHERE weight  >  10"
normalized_query = normalize_query(query)
print(normalized_query)


select count(*) from pets where weight > 10


In [33]:
for createContextItem in sqlCreateContextPoints:
    if "answer" in createContextItem:
        # print(item1["answer"])
        createContextItem["normalised"] = normalize_query(createContextItem["answer"])

for spiderItem in spiderDataPoints:
    if "query" in spiderItem:
        # print(item2["query"])
        spiderItem["normalised"] = normalize_query(spiderItem["query"])

In [34]:
sqlCreateContextPoints

[{'question': 'How many heads of the departments are older than 56 ?',
  'context': 'CREATE TABLE head (age INTEGER)',
  'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
  'normalised': 'select count(*) from head where age > 56'},
 {'question': 'List the name, born state and age of the heads of departments ordered by age.',
  'context': 'CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR)',
  'answer': 'SELECT name, born_state, age FROM head ORDER BY age',
  'normalised': 'select name, born_state, age from head order by age'},
 {'question': 'List the creation year, name and budget of each department.',
  'context': 'CREATE TABLE department (creation VARCHAR, name VARCHAR, budget_in_billions VARCHAR)',
  'answer': 'SELECT creation, name, budget_in_billions FROM department',
  'normalised': 'select creation, name, budget_in_billions from department'},
 {'question': 'What are the maximum and minimum budget of the departments?',
  'context': 'CREATE TABLE department (bud

In [35]:


# Step 2 and 3: Iterate over each object in the second JSON file, 
# extract the "query" attribute, and find the matching object in the first JSON file
new_list = []
for spiderItem in spiderDataPoints:
    spiderItemNormalised = spiderItem.get("normalised")
    spiderQuestion = spiderItem.get("question")
    # print(query2)
    foundQuery2 = False
    for createContextItem in sqlCreateContextPoints:
        createContextItemNormalised = createContextItem.get("normalised")
        createContextQuestion = createContextItem.get("question")
        # print(answer1)
        if (spiderItemNormalised == createContextItemNormalised or spiderQuestion == createContextQuestion):
            createContextItem["db_id"] = spiderItem["db_id"]
            createContextItem["question"] = spiderItem["question"]
            createContextItem["answer"] = spiderItem["query"]

            print(createContextItem["db_id"])
            new_list.append(createContextItem)
            sqlCreateContextPoints.remove(createContextItem) 
            foundQuery2 = True
            break
    if not foundQuery2:
        print("Could not find a match for: " + spiderItemNormalised + " " + spiderQuestion)



department_management
department_management
department_management
department_management
department_management
department_management
department_management
department_management
department_management
department_management
department_management
department_management
department_management
department_management
department_management
department_management
farm
Could not find a match for: select count(*) from farm Count the number of farms.
farm
Could not find a match for: select total_horses from farm order by total_horses asc What is the total horses record for each farm, sorted ascending?
farm
Could not find a match for: select hosts from farm_competition where theme != 'aliens' Return the hosts of competitions for which the theme is not Aliens?
farm
Could not find a match for: select theme from farm_competition order by year asc Return the themes of farm competitions, sorted by year ascending.
farm
Could not find a match for: select avg(working_horses) from farm where total_horses > 5000 

In [36]:
print(len(new_list))
print(len(spiderDataPoints))


3961
7000


In [37]:
def remove_normalised_property(obj_list):
    for obj in obj_list:
        if "normalised" in obj:
            del obj["normalised"]
    return obj_list

In [38]:
result_list = remove_normalised_property(new_list)
print(result_list[0])


{'question': 'How many heads of the departments are older than 56 ?', 'context': 'CREATE TABLE head (age INTEGER)', 'answer': 'SELECT count(*) FROM head WHERE age  >  56', 'db_id': 'department_management'}


In [39]:
print(len(new_list))
print(len(spiderDataPoints))


3961
7000


In [40]:
# Step 4: Save the new list to a new JSON file
with open('spider_create_context_train_db_id.json', 'w') as f:
    json.dump(result_list, f, indent=4)

In [64]:
from datasets import load_dataset
import json
import os

# Load the dataset from Hugging Face
dataset = load_dataset('lamini/spider_text_to_sql')

# Create a directory to store the JSON files
os.makedirs('lamini', exist_ok=True)

# Convert the dataset splits to lists of dictionaries
train_data = [dict(item) for item in dataset['train']]
validation_data = [dict(item) for item in dataset['validation']]

# Write the train split to a JSON file
with open('lamini/train.json', 'w') as train_file:
    json.dump(train_data, train_file, ensure_ascii=False, indent=4)

# Write the validation split to a JSON file
with open('lamini/validation.json', 'w') as validation_file:
    json.dump(validation_data, validation_file, ensure_ascii=False, indent=4)

print("Dataset has been downloaded and saved to 'lamini' directory.")


Dataset has been downloaded and saved to 'lamini' directory.
